# **遷移式學習** -站在巨人的肩膀上的深度學習

## 1. 人工智慧的問題解決方法: CRISP-DM 框架
<img src="https://www.tribloom.com/wp-content/uploads/2019/08/CRISP-DM_Process_Diagram-768x769.png" height=500>

Follow the CRSIP-DM method
1. Step 1: Import library, import data
2. Step 2: Pre-processing (missing data, categorical type, normalization, format transform)
3. Step 3: Build ML Model
4. Step 4: Evaluate Model
5. Step 5: Deploy (Prediction)

# Step 1: Import library and Loading Data

## Step 1.1 install package 

In [ ]:
!pip3 install torch torchvision
!pip3 install gradio
!pip install Pillow


## Step 1-2: 匯入套件



In [ ]:
import gradio as gr
from torchvision import datasets, transforms, models
import torch
import matplotlib.pyplot as plt
import numpy as np

## Step 1-3: 使用GPU on Colab
* device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
* model.to(device)
* inputs = inputs.to(device)
* labels = labels.to(device)



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## Step 1-4 Loading data from github with folders as labels

* ant and bee github <br>
!git clone https://github.com/jaddoescad/ants_and_bees.git

* medical mask github <br>
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git


In [ ]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

## Step 1-5 確認目錄
* with_mask 是 Label =0
* without_mask 是 Label=1

In [ ]:
!ls ./Face-Mask-Detection/dataset/

# Step 2: Data Preprocessing


## Step 2-1 資料格式轉換
* 將資料做一些normalization 以增強深度學習模型效能
* 將資料做一些格式轉換以符合深度學習模型的輸入格式 Tensor

In [ ]:
transform_train = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                      transforms.ColorJitter(brightness=1, contrast=1, saturation=1),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])


transform = transforms.Compose([transforms.Resize((224,224)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])

training_dataset = datasets.ImageFolder('Face-Mask-Detection/dataset/', transform=transform_train)
validation_dataset = datasets.ImageFolder('Face-Mask-Detection/dataset/', transform=transform)

training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=20, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size = 20, shuffle=False)

In [ ]:
validation_dataset = datasets.ImageFolder('Face-Mask-Detection/dataset/', transform=transform)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size = 20, shuffle=False)

print(len(training_dataset))
print(len(validation_dataset))

## Step 2-2 觀察一下影像資料

In [ ]:
def im_convert(tensor):
  image = tensor.cpu().clone().detach().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

classes=('mask', 'no_mask')

dataiter = iter(training_loader)
images,labels = dataiter.next()
fig = plt.figure(figsize=(25, 4))

for idx in np.arange(20):
  ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
  plt.imshow(im_convert(images[idx]))
  ax.set_title(classes[labels[idx].item()])

# Step 3: Build Model
*使用pretraind vgg16模型

## Step 3-1: loading torchvision的 models 預先triain好的模型
* 所有pre-trained models in torchvision

https://pytorch.org/vision/0.8/models.html

* AlexNet
* VGG
* ResNet
* SqueezeNet
* DenseNet
* Inception v3
* GoogLeNet
* ShuffleNet v2
* MobileNet v2
* ResNeXt
* Wide ResNet
* MNASNet

## Step 3-2 各經典模型summary <br>
https://ithelp.ithome.com.tw/articles/10192162

<img src="https://ithelp.ithome.com.tw/upload/images/20171206/200019764r3qCPSJxX.png" height=800, width=300>Vgg16</img>

圖. VGG16 結構圖，圖片來源：Building powerful image classification models using very little data



## Step 3-3  使用vgg16 pretrain 模型

In [ ]:
model = models.vgg16(pretrained=True)

## Step 3-4 把梯度更新全部關閉, 只留後面三級分類層 (vgg16/vgg19示意圖)
 
<img src="https://ithelp.ithome.com.tw/upload/images/20171206/20001976yeCo1PvEOs.jpg" width=500 />

In [ ]:

for param in model.features.parameters():
  param.requires_grad = False

print(model)

## Step 3-5 修改模型讓最後輸出兩類

In [ ]:
import torch.nn as nn
n_inputs = model.classifier[6].in_features
last_layer = nn.Linear(n_inputs, len(classes)) # 2 classes 最後一級
model.classifier[6] = last_layer
model.to(device)
print(model)

## Step 3-7 Training Model

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

epochs = 3
running_loss_history = []
running_corrects_history = []
val_running_loss_history = []
val_running_corrects_history = []

for e in range(epochs):
  
  running_loss = 0.0
  running_corrects = 0.0
  val_running_loss = 0.0
  val_running_corrects = 0.0
  
  for inputs,labels in training_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _, preds = torch.max(outputs, 1)
    running_loss += loss.item()
    running_corrects += torch.sum(preds == labels.data)

  else:
    with torch.no_grad():
      for val_inputs, val_labels in validation_loader:
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_labels)
        
        _, val_preds = torch.max(val_outputs, 1)
        val_running_loss += val_loss.item()
        val_running_corrects += torch.sum(val_preds == val_labels.data)
      
    epoch_loss = running_loss/len(training_loader.dataset)
    epoch_acc = running_corrects.float()/ len(training_loader.dataset)
    running_loss_history.append(epoch_loss)
    running_corrects_history.append(epoch_acc.item())
    
    val_epoch_loss = val_running_loss/len(validation_loader.dataset)
    val_epoch_acc = val_running_corrects.float()/ len(validation_loader.dataset)
    val_running_loss_history.append(val_epoch_loss)
    val_running_corrects_history.append(val_epoch_acc.item())
    print('epoch :', (e+1))
    print('training loss: {:.4f}, acc {:.4f} '.format(epoch_loss, epoch_acc.item()))
    print('validation loss: {:.4f}, validation acc {:.4f} '.format(val_epoch_loss, val_epoch_acc.item()))

:# Step 4: Evaluate Model and See Results

In [ ]:
plt.plot(running_loss_history, label='training loss')
plt.plot(val_running_loss_history, label='validation loss')
plt.legend()

In [ ]:
plt.plot(running_corrects_history, label='training accuracy')
plt.plot(val_running_corrects_history, label='validation accuracy')
plt.legend()

# Step 5: Deploy Model for Practical Use
# Make an AI App

In [ ]:
# !pip3 install pillow==4.0.0
import PIL.ImageOps

In [ ]:


import requests
from PIL import Image

url = 'http://media.rojaklah.com/wp-content/uploads/2017/09/19150232/1909bigstar1.jpg'

response = requests.get(url, stream = True)
img = Image.open(response.raw)
plt.imshow(img)


In [ ]:

img = transform(img) 
plt.imshow(im_convert(img))

In [ ]:
image = img.to(device).unsqueeze(0)
output = model(image)
_, pred = torch.max(output, 1)
print(classes[pred.item()])

In [ ]:
import requests
from PIL import Image

url2='http://i1.kknews.cc/L4QL2XatiKQt9Z6yC-ttG4KM-Gk48BY/0.jpg'
response = requests.get(url2, stream = True)
img = Image.open(response.raw)
plt.imshow(img)
img = transform(img)
plt.imshow(im_convert(img))
image = img.to(device).unsqueeze(0)
output = model(image)
_, pred = torch.max(output, 1)
print(classes[pred.item()])

In [ ]:
print(classes[pred.item()])


## 特別介紹 gradio :https://www.gradio.app/ <br>
Gradio is the fastest way to demo your machine learning model with a friendly web interface so that anyone can use it, anywhere!

In [ ]:
def predict(img):
    labels = ['mask', 'no_mask']
    img = transform(img)
    image = img.to(device).unsqueeze(0)
    output = model(image)
    _, pred = torch.max(output, 1)
    return labels[pred.item()]

gr.Interface(fn=predict, 
             inputs=gr.Image(type="pil"),
             outputs='label').launch(debug=True, show_error=True)